In [11]:
import os
import cv2
import pickle
import argparse
import numpy as np
from imutils import paths
from PIL import Image
from keras.models import load_model
from keras.models import model_from_json
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.optimizers import Adam
from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

In [12]:
#ap = argparse.ArgumentParser()
#ap.add_argument("-m", "--model", required=True, help="json file with trained model to use")
#ap.add_argument("-w", "--weights", required=True, help="h5 file with trained model to use")
#ap.add_argument("-l", "--labels", required=True, help="image labels")
#ap.add_argument("-i", "--image", required=True, help="image to analyse")
#args = vars(ap.parse_args())
args = {}
args["model"] = "C:/Users/UTILISATEUR/Sandbox/catfaces/notebooks/cat_model_architecture.json"
args["weights"] = "C:/Users/UTILISATEUR/Sandbox/catfaces/notebooks/cat_neural_model.h5"
args["labels"] = "C:/Users/UTILISATEUR/Sandbox/catfaces/notebooks/cat_labels.bin"
args["images"] = "C:/Users/UTILISATEUR/Sandbox/catfaces/testing"

In [13]:
#with open(args["model"], 'r') as f:
	#model = model_from_json(f.read())

#model .load_weights(args["weights"])
#lb = pickle.loads(open(args["labels"], "rb").read())

print("[INFO] loading images...")
imagePaths = paths.list_images(args["images"])
data = []
labels = []

[INFO] loading images...


In [23]:
# loop over our input images
for imagePath in imagePaths:
	# load the input image from disk, resize it to 32x32 pixels, scale
	# the pixel intensities to the range [0, 1], and then update our
	# images list
	image = Image.open(imagePath)
	image = np.array(image.resize((224, 224))) / 255.0
	image = image.astype("float") / 255.0
	image = img_to_array(image)
	image = np.expand_dims(image, axis=0)
	data.append(image)
 
	# extract the class label from the file path and update the
	# labels list
	label = imagePath.split(os.path.sep)[-2]
	labels.append(label)

In [24]:
#(testX, testY) = (np.array(data),np.array(labels))
(trainX, testX, trainY, testY) = train_test_split(np.array(data),np.array(labels), test_size=0.99)

In [27]:
with open(args["model"], 'r') as f:
	model = model_from_json(f.read())

model.load_weights(args["weights"])
lb = pickle.loads(open(args["labels"], "rb").read())
#lb = LabelBinarizer()
#labels = lb.fit_transform(labels)

In [18]:
#image = cv2.imread(args["image"])
#image = cv2.resize(image, (224, 224))
#image = image.astype("float") / 255.0
#image = img_to_array(image)
#image = np.expand_dims(image, axis=0)

In [28]:
#probas = model.predict(image)[0]

#idx = np.argmax(probas)
#label = lb.classes_[idx]
#proba = probas[idx]

#print("Cat: " + label + " with probability:" + str(proba))
# evaluate the network
print("[INFO] evaluating network...")
predictions = model.predict(testX, batch_size=1)
print(classification_report(testY.argmax(axis=1), predictions.argmax(axis=1), target_names=lb.classes_))

[INFO] evaluating network...
             precision    recall  f1-score   support

     bagguy       0.85      0.68      0.76        25
      felix       0.95      0.76      0.84        25
     gritus       0.94      0.68      0.79        25
    grokado       0.75      0.75      0.75         4
      kipik       0.00      0.00      0.00        24
     mikado       0.00      0.00      0.00        25
      nocat       0.00      0.00      0.00         5
        ota       0.00      0.00      0.00         0
    triknik       0.00      0.00      0.00         0

avg / total       0.54      0.42      0.47       133



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
